## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.
C:\Users\ian\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ian\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ian\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ian\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Fu

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [18]:
from keras.regularizers import l1, l2, l1_l2

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l1_ratio=0.0, l2_ratio=0.0):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.models.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i==0:
            x = keras.layers.Dense(units=n_units, 
                                   activation='relu', 
                                   name='hidden_layer'+str(i+1),
                                  kernel_regularizer=l1_l2(l1_ratio,l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation='relu', 
                                   name='hidden_layer'+str(i+1),
                                  kernel_regularizer=l1_l2(l1_ratio,l2_ratio))(x)
    out = keras.layers.Dense(units=output_units, activation='softmax', name='output_layer')(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    
    return model

In [16]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
L1_EXP = [1e-2, 1e-4, 1e-8, 0.0]
L2_EXP = [1e-2, 1e-4, 1e-8, 0.0]

In [20]:
import itertools
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

for l1r, l2r in itertools.product(L1_EXP,L2_EXP):
    keras.backend.clear_session()
    print('Experiment with L1 = %.6f, L2 = %6.f'%(l1r , l2r))
    model = build_mlp(input_shape=x_train.shape[1:], l1_ratio=l1r, l2_ratio=l2r)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=optimizer)
    model.fit(x_train,y_train,
             epochs=EPOCHS,
             batch_size=BATCH_SIZE,
             validation_data=(x_test, y_test),
             shuffle=True)
    
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l1-%s-l2-%s" % (str(l1r),str(l2r))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with L1 = 0.010000, L2 =      0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output_layer (Dense)         (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 13s 261us/step - loss: 198.6737 - acc: 0.2393 - val_loss: 41.1233 - val_acc: 0.2730
Epoch 2/50
50000/50000 [==============================] - 13s 252us/step - loss: 19.1190 - acc: 0.1193 - val_loss: 7.2755 - val_acc: 0.1000
Epoch 3/50
50000/50000 [==============================] - 12s 247us/step - loss: 4.1047 - acc: 0.1000 - val_loss: 2.6482 - val_acc: 0.1000
Epoch 4/50
50000/50000 [==============================] - 13s 252us/step - loss: 2.4862 - acc: 0.0989 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 5/50
50000/50000 [==============================] - 13s 254us/step - loss: 2.4626 - acc: 0.0979 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 6/50
50000/50000 [==============================] - 13s 254us/step - loss: 2.4626 - acc: 0.0974 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 7/50
50000/50000 [==============================] - 13s 253us/step - loss: 2.4626 - acc: 0.0971 - val_loss

50000/50000 [==============================] - 13s 262us/step - loss: 198.8025 - acc: 0.2274 - val_loss: 41.1282 - val_acc: 0.2498
Epoch 2/50
50000/50000 [==============================] - 13s 256us/step - loss: 19.1054 - acc: 0.1167 - val_loss: 7.2721 - val_acc: 0.1000
Epoch 3/50
50000/50000 [==============================] - 13s 253us/step - loss: 4.1020 - acc: 0.0979 - val_loss: 2.6478 - val_acc: 0.1000
Epoch 4/50
50000/50000 [==============================] - 13s 253us/step - loss: 2.4863 - acc: 0.0989 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 5/50
50000/50000 [==============================] - 12s 240us/step - loss: 2.4626 - acc: 0.0961 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 6/50
50000/50000 [==============================] - 12s 236us/step - loss: 2.4626 - acc: 0.0987 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 7/50
50000/50000 [==============================] - 12s 241us/step - loss: 2.4626 - acc: 0.0978 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 8/50
50000/50000 [==========

Epoch 2/50
50000/50000 [==============================] - 11s 227us/step - loss: 19.0960 - acc: 0.1143 - val_loss: 7.2904 - val_acc: 0.1000
Epoch 3/50
50000/50000 [==============================] - 12s 230us/step - loss: 4.1121 - acc: 0.0997 - val_loss: 2.6457 - val_acc: 0.1000
Epoch 4/50
50000/50000 [==============================] - 12s 230us/step - loss: 2.4854 - acc: 0.0996 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 5/50
50000/50000 [==============================] - 12s 231us/step - loss: 2.4626 - acc: 0.0980 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 6/50
50000/50000 [==============================] - 11s 226us/step - loss: 2.4626 - acc: 0.0987 - val_loss: 2.4626 - val_acc: 0.1000
Epoch 7/50
50000/50000 [==============================] - 12s 232us/step - loss: 2.4626 - acc: 0.0977 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 8/50
50000/50000 [==============================] - 11s 229us/step - loss: 2.4626 - acc: 0.0977 - val_loss: 2.4625 - val_acc: 0.1000
Epoch 9/50
50000/50000 [==

50000/50000 [==============================] - 17s 341us/step - loss: 16.1433 - acc: 0.3579 - val_loss: 14.9812 - val_acc: 0.3747
Epoch 3/50
50000/50000 [==============================] - 16s 330us/step - loss: 13.9543 - acc: 0.3818 - val_loss: 12.9766 - val_acc: 0.3786
Epoch 4/50
50000/50000 [==============================] - 18s 352us/step - loss: 12.1041 - acc: 0.3950 - val_loss: 11.2684 - val_acc: 0.4027
Epoch 5/50
50000/50000 [==============================] - 15s 306us/step - loss: 10.5338 - acc: 0.4035 - val_loss: 9.8228 - val_acc: 0.4123
Epoch 6/50
50000/50000 [==============================] - 14s 274us/step - loss: 9.1963 - acc: 0.4105 - val_loss: 8.5914 - val_acc: 0.4173
Epoch 7/50
50000/50000 [==============================] - 14s 273us/step - loss: 8.0581 - acc: 0.4170 - val_loss: 7.5416 - val_acc: 0.4201
Epoch 8/50
50000/50000 [==============================] - 14s 271us/step - loss: 7.0882 - acc: 0.4202 - val_loss: 6.6513 - val_acc: 0.4242
Epoch 9/50
50000/50000 [=======

...

50000/50000 [==============================] - 13s 265us/step - loss: 13.0134 - acc: 0.3589 - val_loss: 12.1212 - val_acc: 0.3694
Epoch 3/50
50000/50000 [==============================] - 13s 262us/step - loss: 11.3420 - acc: 0.3816 - val_loss: 10.5946 - val_acc: 0.3857
Epoch 4/50
50000/50000 [==============================] - 13s 266us/step - loss: 9.9310 - acc: 0.3941 - val_loss: 9.2900 - val_acc: 0.4068
Epoch 5/50
50000/50000 [==============================] - 13s 265us/step - loss: 8.7304 - acc: 0.4048 - val_loss: 8.1878 - val_acc: 0.4137
Epoch 6/50
50000/50000 [==============================] - 13s 262us/step - loss: 7.7088 - acc: 0.4117 - val_loss: 7.2445 - val_acc: 0.4181
Epoch 7/50
50000/50000 [==============================] - 13s 269us/step - loss: 6.8369 - acc: 0.4190 - val_loss: 6.4492 - val_acc: 0.4151
Epoch 8/50
50000/50000 [==============================] - 14s 282us/step - loss: 6.0920 - acc: 0.4261 - val_loss: 5.7532 - val_acc: 0.4297
Epoch 9/50
50000/50000 [==========

50000/50000 [==============================] - 13s 267us/step - loss: 1.9583 - acc: 0.3607 - val_loss: 1.9192 - val_acc: 0.3590
Epoch 3/50
50000/50000 [==============================] - 14s 282us/step - loss: 1.8726 - acc: 0.3929 - val_loss: 1.8400 - val_acc: 0.4049
Epoch 4/50
50000/50000 [==============================] - 14s 281us/step - loss: 1.8111 - acc: 0.4151 - val_loss: 1.7885 - val_acc: 0.4248
Epoch 5/50
50000/50000 [==============================] - 14s 273us/step - loss: 1.7624 - acc: 0.4310 - val_loss: 1.7449 - val_acc: 0.4397
Epoch 6/50
50000/50000 [==============================] - 13s 261us/step - loss: 1.7200 - acc: 0.4472 - val_loss: 1.7114 - val_acc: 0.4472
Epoch 7/50
50000/50000 [==============================] - 13s 263us/step - loss: 1.6862 - acc: 0.4588 - val_loss: 1.6884 - val_acc: 0.4600
Epoch 8/50
50000/50000 [==============================] - 13s 265us/step - loss: 1.6539 - acc: 0.4697 - val_loss: 1.6621 - val_acc: 0.4619
Epoch 9/50
50000/50000 [==============

50000/50000 [==============================] - 14s 278us/step - loss: 1.8165 - acc: 0.3613 - val_loss: 1.7727 - val_acc: 0.3695
Epoch 3/50
50000/50000 [==============================] - 14s 289us/step - loss: 1.7309 - acc: 0.3906 - val_loss: 1.7080 - val_acc: 0.3966
Epoch 4/50
50000/50000 [==============================] - 13s 263us/step - loss: 1.6714 - acc: 0.4117 - val_loss: 1.6576 - val_acc: 0.4175
Epoch 5/50
50000/50000 [==============================] - 13s 264us/step - loss: 1.6261 - acc: 0.4292 - val_loss: 1.6193 - val_acc: 0.4324
Epoch 6/50
50000/50000 [==============================] - 13s 262us/step - loss: 1.5882 - acc: 0.4429 - val_loss: 1.5816 - val_acc: 0.4415
Epoch 7/50
50000/50000 [==============================] - 14s 275us/step - loss: 1.5570 - acc: 0.4524 - val_loss: 1.5566 - val_acc: 0.4510
Epoch 8/50
50000/50000 [==============================] - 13s 265us/step - loss: 1.5239 - acc: 0.4640 - val_loss: 1.5331 - val_acc: 0.4598
Epoch 9/50
50000/50000 [==============

Epoch 3/50
50000/50000 [==============================] - 12s 246us/step - loss: 1.7165 - acc: 0.4009 - val_loss: 1.6821 - val_acc: 0.4120
Epoch 4/50
50000/50000 [==============================] - 12s 241us/step - loss: 1.6577 - acc: 0.4198 - val_loss: 1.6370 - val_acc: 0.4228
Epoch 5/50
50000/50000 [==============================] - 12s 240us/step - loss: 1.6102 - acc: 0.4364 - val_loss: 1.5989 - val_acc: 0.4351
Epoch 6/50
50000/50000 [==============================] - 12s 238us/step - loss: 1.5714 - acc: 0.4492 - val_loss: 1.5672 - val_acc: 0.4465
Epoch 7/50
50000/50000 [==============================] - 11s 228us/step - loss: 1.5363 - acc: 0.4628 - val_loss: 1.5530 - val_acc: 0.4514
Epoch 8/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.5072 - acc: 0.4731 - val_loss: 1.5250 - val_acc: 0.4593
Epoch 9/50
50000/50000 [==============================] - 12s 236us/step - loss: 1.4809 - acc: 0.4816 - val_loss: 1.4919 - val_acc: 0.4682
Epoch 10/50
50000/50000 [==

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""